In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import copy

## Data Sources
There are two similar yet slightly different sources for this data. The 2000 data comes from the Decennial United States Census whereas the 2009-2016 data comes from American Community Survey 5-year estimates. Both data sources contain roughly the same features, format, and geographic contraints (census tracts). 

In [2]:
#system agnostic data path
raw_data_dir = os.path.join(os.getcwd(), 'data', 'raw')
#empty dict to hold all the variables, this will come in handy later
empty_vars = {'income': '', 'education': '', 'race': '',
              'rent': '', 'value': '', 'unemployment': ''}
#these are the data variable names for the decennial census
dec_data_vars = {'P052': 'income', 'QTP20': 'education', 'P007': 'race',
                 'H062': 'rent', 'H084': 'value', 'QTP24': 'unemployment'}
#these are the data variable names for the ACS 5-year estimates
acs_data_vars = {'B19001': 'income', 'S1501': 'education', 'B02001': 'race',
                 'B25063': 'rent', 'B25075': 'value', 'S2301': 'unemployment'}
#this will hold all the different file names to make reading data easier
data_dict = {
    '2000': empty_vars.copy(),
    '2009': empty_vars.copy(),
    '2010': empty_vars.copy(),
    '2011': empty_vars.copy(),
    '2012': empty_vars.copy(),
    '2013': empty_vars.copy(),
    '2014': empty_vars.copy(),
    '2015': empty_vars.copy(),
    '2016': empty_vars.copy()
}
#loop through raw data files (csv only) and assign each one to proper key
for fil in os.listdir(raw_data_dir):
    if fil.endswith('.csv'):
        fil_spl = fil.split('_')
        if fil_spl[3] in acs_data_vars.keys():
            data_dict['20'+fil_spl[1]][acs_data_vars[fil_spl[3]]] = os.path.join(raw_data_dir, fil)
        elif fil_spl[3] in dec_data_vars.keys():
            data_dict['20'+fil_spl[1]][dec_data_vars[fil_spl[3]]] = os.path.join(raw_data_dir, fil)
        else:
            print('Unexpected file not processed: {}'.format(fil))

Unexpected file not processed: ACS_09_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_10_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_11_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_12_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_13_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_14_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_15_5YR_B11001_with_ann.csv
Unexpected file not processed: ACS_16_5YR_B11001_with_ann.csv
Unexpected file not processed: DEC_00_SF4_QTP10_with_ann.csv


## Relating 2000 and 2010 Census Tracts
One complication that arises while working with census data is that tracts change over time as population changes. Tracts generally encompass a population between 2,500 and 8,000 people and are the intended to be a rough equivalent of a neighborhood. As the population grows, shrinks, and moves, some census tracts are split into smaller tracts while others are merged to form a new tract. This is the case between the 2000-2009 data, which use the 2000 census tracts, vs the 2010-2016 data, which use the 2010 census tracts. 

Thankfully, the Census Bureau tracks this sort of change very carefully and presents the morphology of census tracts in a detailed relationship file.

In [3]:
geoid_2000 = pd.read_csv(data_dict['2000']['income'], skiprows=1)['Id2']
geoid_2010 = pd.read_csv(data_dict['2016']['income'], skiprows=1)['Id2']

removed = list(set(geoid_2000) - set(geoid_2010))
added = list(set(geoid_2010) - set(geoid_2000))

tract_relations = pd.read_csv(os.path.join(os.getcwd(), 'census_tract_shapefile', 'census_tract_relation_file.csv'),
                              usecols=[1, 3, 12, 25, 26])
tract_relations = tract_relations[tract_relations['county00'].isin([21, 55, 209, 453, 491])]

need_to_change = tract_relations[tract_relations['geoid00'].isin(removed)]
need_to_change = need_to_change[need_to_change['geoid10'].isin(added)]
need_to_change = need_to_change[need_to_change['poppct00'] > 0]
need_to_change = need_to_change[need_to_change['poppct10'] > 1]

need_to_change.to_csv(os.path.join(os.getcwd(), 'census_tract_shapefile', 'need_to_change.csv'))

merged = need_to_change[(need_to_change['poppct00'] > 95) & (need_to_change['poppct10'] < 100)]
split = need_to_change[need_to_change['poppct10'] > 95]

In [4]:
def fix_tracts(df):
    #the tract for austin-bergstrom airport is missing from 2000 and 2009
    #there's no population or housing units in the tract (geoid: 48453980000)
    #so it can be filled in with zeros
    df.loc[48453980000] = 0
    #merged
    for geoid in merged['geoid10']:
        df.loc[geoid] = df.loc[list(merged['geoid00'])].sum()
    df.drop(list(merged['geoid00']), inplace=True)
    #split
    for row in split.iterrows():
        df.loc[row[1]['geoid10']] = round(df.loc[row[1]['geoid00']] * row[1]['poppct00'] / 100, 0)
    df.drop(list(split['geoid00'].unique()), inplace=True)
    df.astype('int')
    
    return df

In [5]:
def read_income(year):
    if year == '2000':
        usecols = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
                   12, 13, 14, 15, 16, 17, 18, 19]
    else:
        usecols=[1, 3, 5, 7, 9, 11, 13, 15, 17, 19,
                 21, 23, 25, 27, 29, 31, 33, 35]
    income = pd.read_csv(data_dict[year]['income'], skiprows=1, usecols=usecols)
    income.index = income.pop('Id2')
    income.index.name = 'geoid'
    income.columns = ['total', '<10k', '[10k-15k)', '[15k-20k)',
                      '[20k-25k)', '[25k-30k)', '[30k-35k)', '[35k-40k)', '[40k-45k)',
                      '[45k-50k)', '[50k-60k)', '[60k-75k)', '[75k-100k)', '[100k-125k)',
                      '[125k-150k)', '[150k-200k)', '>200k']
    if year in ['2000', '2009']:
        income = fix_tracts(income)
    
    return income

In [6]:
def read_education(year):
    if year == '2000':
        usecols = [3, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53]
    elif year in ['2009', '2010', '2011', '2012', '2013']:
        usecols = [1, 33, 39, 45, 51, 57, 63, 69, 75]
    elif year == '2014':
        usecols = [1, 8, 9, 10, 11, 12, 13, 14, 15]
    else:
        usecols = [1, 13, 15, 17, 19, 21, 23, 25, 27]
    
    education = pd.read_csv(data_dict[year]['education'], skiprows=1, usecols=usecols, na_values=['-'])
    education.index = education.pop('Id2')
    education.index.name = 'geoid'
    
    if year == '2000':
        education.columns = ['total', '<5', '5-8', '9-12', 'high_school', 'some_college_1', 
                             'some_college_2', 'associate', 'bachelor', 'master', 
                             'professional', 'doctorate']
        education['<9'] = education.pop('<5') + education.pop('5-8')
        education['some_college'] = education.pop('some_college_1') + education.pop('some_college_2')
        education['graduate'] = education.pop('master') + education.pop('professional') \
                                + education.pop('doctorate')
        education = education[['total', '<9', '9-12', 'high_school', 'some_college',
                               'associate', 'bachelor', 'graduate']]
    
    elif year in ['2015', '2016']:
        education.columns = ['total', '<9', '9-12', 'high_school', 'some_college',
                             'associate', 'bachelor', 'graduate']
    
    else:
        education.columns = ['total', '<9', '9-12', 'high_school', 'some_college',
                             'associate', 'bachelor', 'graduate']
        education.fillna(0, inplace=True) #not actually missing - Austin-Bergrstrom Intl. Airport
        perc_cols = education.columns[1:]
        for c in perc_cols:
            education[c] = round(education[c] / 100 * education['total'], 0).astype('int')
        
    if year in ['2000', '2009']:
        education = fix_tracts(education)
        
    return education

In [7]:
def read_race(year):
    if year == '2000':
        usecols = [1, 3, 4]
    else:
        usecols = [1, 3, 5]
    race = pd.read_csv(data_dict[year]['race'], skiprows=1, usecols=usecols)
    race.index = race.pop('Id2')
    race.index.name = 'geoid'
    race.columns = ['total_population', 'white_alone']
    
    if year in ['2000', '2009']:
        race = fix_tracts(race)
        
    '''
    still need to convert to percent (here or next step?)
    '''
    
    return race

In [8]:
def read_families(year):
    if year == '2000':
        usecols = [3, 5, 7]
    else:
        usecols = [1, 3, 5]
    families = pd.read_csv(data_dict[year]['families'], skiprows=1, usecols=usecols)
    families.index = families.pop('Id2')
    families.index.name = 'geoid'
    families.columns = ['total_households', 'family_households']
    
    if year in ['2000', '2009']:
        families = fix_tracts(families)
        
    '''
    still need to convert to percent (here or next step?)
    '''
    
    return families

In [9]:
def read_rent(year):
    if year == '2000':
        usecols = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 
                   17, 18, 19, 20, 21, 22, 23, 24, 25]
    elif year in ['2009', '2010', '2011', '2012', '2013', '2014']:
        usecols = [1, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 
                   27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47]
    else:
        usecols=[1, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 
                 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 
                 51, 53]
    rent = pd.read_csv(data_dict[year]['rent'], skiprows=1, usecols=usecols)
    rent.index = rent.pop('Id2')
    rent.index.name = 'geoid'
    if year in ['2015', '2016']:
        rent['>2000'] = rent[rent.columns[21:25]].sum(axis=1)
        rent.drop(rent.columns[[21, 22, 23, 24]], axis=1, inplace=True)
    
    rent.columns = ['total', '<100', '[100-150)', '[150-200)', '[200-250)',
                    '[250-300)', '[300-350)', '[350-400)', '[400-450)', '[450-500)', '[500-550)',
                    '[550-600)', '[600-650)', '[650-700)', '[700-750)', '[750-800)', '[800-900)',
                    '[900-1000)', '[1000-1250)', '[1250-1500)', '[1500-2000)', '>2000']
    
    if year in ['2000', '2009']:
        rent = fix_tracts(rent)
    
    return rent

In [10]:
def read_value(year):
    if year == '2000':
        usecols = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 
                   16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
    elif year in ['2009', '2010', '2011', '2012', '2013', '2014']:
        usecols = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 
                   29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51]
    else:
        usecols = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 
                   29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55]
    
    value = pd.read_csv(data_dict[year]['value'], skiprows=1, usecols=usecols)
    value.index = value.pop('Id2')
    value.index.name = 'geoid'
    
    if year in ['2015', '2016']:
        value['>1M'] = value[value.columns[25:28]].sum(axis=1)
        value.drop(value.columns[[25, 26, 27]], axis=1, inplace=True)
    
    value.columns = ['total', '<10k', '[10k-15k)', '[15k-20k)', '[20k-25k)', 
                     '[25k-30k)', '[30k-35k)', '[35k-40k)', '[40k-50k)', '[50k-60k)', '[60k-70k)', 
                     '[70k-80k)', '[80k-90k)', '[90k-100k)', '[100k-125k)', '[125k-150k)', '[150k-175k)', 
                     '[175k-200k)', '[200k-250k)', '[250k-300k)', '[300k-400k)', '[400k-500k)', 
                     '[500k-750k)', '[750k-1M)', '>1M']
    
    if year in ['2000', '2009']:
        value = fix_tracts(value)
    
    return value

In [11]:
def read_unemployment(year):
    if year == '2000':
        usecols = [3, 5, 35]
    else:
        usecols = [1, 3, 9]
    unemployment = pd.read_csv(data_dict[year]['unemployment'], skiprows=1, usecols=usecols, na_values='-')
    unemployment.index = unemployment.pop('Id2')
    unemployment.index.name = 'geoid'
    unemployment.columns = ['total_in_labor_force', 'unemployed']
    if year != '2000':
        unemployment.fillna(0, inplace=True)
        unemployment['unemployed'] = round(unemployment['total_in_labor_force']\
                                           * unemployment['unemployed'] / 100, 0)
    
    if year in ['2000', '2009']:
        unemployment = fix_tracts(unemployment)
        
    return unemployment

In [12]:
all_data = {
    '2000': empty_vars.copy(),
    '2009': empty_vars.copy(),
    '2010': empty_vars.copy(),
    '2011': empty_vars.copy(),
    '2012': empty_vars.copy(),
    '2013': empty_vars.copy(),
    '2014': empty_vars.copy(),
    '2015': empty_vars.copy(),
    '2016': empty_vars.copy()
}

read_funs = [read_income, read_education, read_race,
             read_rent, read_value, read_unemployment]
read_vars = ['income', 'education', 'race', 
             'rent', 'value', 'unemployment']

for key in all_data.keys():
    for f, v in zip(read_funs, read_vars):
        all_data[key][v] = f(key)
        
pickle_all = open(os.path.join(os.getcwd(), 'data', 'processed', 'all_data.pickle'), 'wb')
pickle.dump(all_data, pickle_all)

#All the data can now be read with the following two lines of code:
#pickle_in = open(os.path.join(os.getcwd(), 'data', 'processed', 'all_data.pickle'), 'rb')
#data = pickle.load(pickle_in)

## Transforming the data
Now we'll aggregate the binned data (rent, value, income, education) into indices, convert race and unemployment to percentages, as well as adjust each year for inflation.

In [13]:
#make a copy of all_data for transforming data 
#need to use copy.deepcopy() here so that the dataframes inside the dictionary are
#copied as well - if you just you all_data.copy(), the dictionary will be copied but
#the dataframes will share memory and any modifications to dataframes will be applied 
#the dataframes in all_data as well
trans_data = copy.deepcopy(all_data)
#the scorer for monetary variables is the middle value of the bin with an assumed
#value for the uppermost bin
income_scorer = np.array([5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5,
                          55, 67.5, 87.5, 112.5, 137.5, 175, 300]) * 1000
rent_scorer = np.array([50, 125, 175, 225, 275, 325, 375, 425, 475, 525, 575,
                        625, 675, 725, 775, 850, 950, 1125, 1375, 1750, 2500])
value_scorer = np.array([5, 12.5, 17.5, 22.5, 27.5, 32.5, 27.5, 45, 55, 65, 75, 
                         85, 95, 112.5, 137.5, 162.5, 187.5, 225, 275, 350, 450, 
                         625, 875, 1500]) * 1000
#education gets a simple 1-8 score
education_scorer = np.arange(1, 8)

#to adjust the values to 2016 dollars, the index are adjusted by the 
#yearly average consumer price index 
inflation_correction = np.array([1.39422697, 1.11865482, 1.10084296, 1.06686838,
                                 1.04536021, 1.03039158, 1.01380104, 1.01263642, 
                                 1.00000000])

for i,year in enumerate(trans_data.keys()):
    for key in trans_data[year].keys():
        tmp = trans_data[year][key]
        #bust out the total population from the race df
        if key == 'race':
            population = tmp['total_population'].to_frame()
        #bust out total in labor force from the unemployment df
        if key == 'unemployment':
            tmp['employed'] = tmp['total_in_labor_force'] - tmp['unemployed']
        #convert columns to percentages of total and drop the total column
        tmp = tmp[tmp.columns[1:]].divide(tmp[tmp.columns[0]], axis=0).fillna(0)
        #calculate the index (single number to represent binned data)
        if len(tmp.columns) > 1:
            if key == 'income':
                tmp['income_index'] = tmp.dot(income_scorer) * inflation_correction[i]
            elif key == 'rent':
                tmp['rent_index'] = tmp.dot(rent_scorer) * inflation_correction[i]
            elif key == 'value':
                tmp['value_index'] = tmp.dot(value_scorer) * inflation_correction[i]
            elif key == 'education':
                tmp['education_index'] = tmp.dot(education_scorer) 
        #put the new percentage column back into the dataframe
        trans_data[year][key] = tmp[tmp.columns[-1]].copy().to_frame()
    #concatenate the summary columns (1 for each dataframe) into a single dataframe
    trans_data[year] = pd.concat([trans_data[year][key] for key in trans_data[year].keys()], axis=1)
    #add a year column in the first position to be used later
    trans_data[year].insert(0, 'year', year)

In [14]:
#now we'll save this version of the data
pickle_trans = open(os.path.join(os.getcwd(), 'data', 'processed', 'trans_data.pickle'), 'wb')
pickle.dump(trans_data, pickle_trans)

In [15]:
#next we'll concatenate all the dataframes in trans_data into a single dataframe
merged_data = pd.concat([trans_data[year] for year in trans_data.keys()])
pickle_merged = open(os.path.join(os.getcwd(), 'data', 'processed', 'merged_data.pickle'), 'wb')
pickle.dump(merged_data, pickle_merged)